# GraphRAG Langchain Integration

## Neo4j graph

In [1]:
import time

import pandas as pd
from neo4j import GraphDatabase

import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

In [2]:
NEO4J_URI = os.getenv("NEO4J_URI")  # or neo4j+s://xxxx.databases.neo4j.io
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE")

### query by direct cypher query

In [3]:
# from langchain_community.graphs import Neo4jGraph

# graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

# QUERY = """
# "MATCH (m:Movie)-[:IN_GENRE]->(:Genre {name:$genre})
# RETURN m.title, m.plot
# ORDER BY m.imdbRating DESC LIMIT 5"
# """

# graph.query(QUERY, genre="action")

## CypherQAChain

The CypherQAChain is a LangChain component that allows you to interact with a Neo4j graph database in natural language. Using an LLM and the graph schema it translates the user question into a Cypher query, executes it against the graph and uses the returned context information and the original question with a second LLM to generate a natural language response.

---

CypherQAChain은 자연어로 Neo4j 그래프 데이터베이스와 상호 작용할 수 있는 LangChain 구성 요소입니다. LLM과 그래프 스키마를 사용하여 사용자 질문을 Cypher 쿼리로 변환하고 그래프에 대해 실행한 다음 반환된 컨텍스트 정보와 원래 질문을 두 번째 LLM과 함께 사용하여 자연어 응답을 생성합니다.

> https://python.langchain.com/v0.2/docs/integrations/graphs/neo4j_cypher/

In [ ]:
# %pip install --upgrade vllm

### Cypher / QA LLM 분리

In [ ]:
# # cypher llm, qa llm을 별도로 놓을 수 있음
# chain = GraphCypherQAChain.from_llm(
#     graph=graph,
#     cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
#     qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
#     verbose=True,
# )

- VLLMOpenAI는 graphcypherqachain 지원 X

In [3]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain_community.llms import VLLMOpenAI

print("## Graph load test ##")
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

# Cypher query to check the number of nodes and relationships
query_nodes = "MATCH (n) RETURN count(n) AS node_count"
query_edges = "MATCH ()-[r]->() RETURN count(r) AS edge_count"

# Execute the queries to get node and edge counts
node_count = graph.query(query_nodes)
edge_count = graph.query(query_edges)

print(f"Number of nodes: {node_count[0]['node_count']}")
print(f"Number of edges: {edge_count[0]['edge_count']}")


## Graph load test ##
Number of nodes: 1968
Number of edges: 1342


In [4]:
graph.schema

'Node properties:\nlaw {metadata_document_title: STRING, metadata_source: STRING, metadata_revise_info: STRING, subtitle: STRING, index: STRING, metadata_date: STRING, content: STRING, metadata_title_doc: STRING, metadata_title_chapter: STRING, metadata_title_section: STRING, metadata_title_subsection: STRING, metadata_title_supplementary: STRING}\nRelationship properties:\n\nThe relationships:\n(:law)-[:REFERS_TO]->(:law)'

### Enhanced schema information

In [5]:
enhanced_graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, enhanced_schema=True)
print(enhanced_graph.schema)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


Node properties:
- **law**
  - `metadata_document_title`: STRING Available options: ['자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 ) ']
  - `metadata_source`: STRING Available options: ['국가법령정보센터']
  - `metadata_revise_info`: STRING Available options: ['법률 제20305호, 2024. 2. 13., 일부개정 ']
  - `subtitle`: STRING Example: "목적"
  - `index`: STRING Example: "제1조"
  - `metadata_date`: STRING Available options: ['시행 2024. 8. 14.']
  - `content`: STRING Example: "이 법은 자본시장에서의 금융혁신과 공정한 경쟁을 촉진하고 투자자를 보호하며 금융투자업을 건"
  - `metadata_title_doc`: STRING Available options: ['제1편 총칙', '제2편 금융투자업', '제3편 증권의 발행 및 유통', '제4편 불공정거래의 규제', '제5편 집합투자기구', '제6편 금융투자업관계기관', '제7편 거래소 <개정 2013. 5. 28.>', '제8편 감독 및 처분', '제9편 보칙', '제10편 벌칙']
  - `metadata_title_chapter`: STRING Example: "제1장 금융투자업의 인가 및 등록"
  - `metadata_title_section`: STRING Example: "제1절 인가요건 및 절차"
  - `metadata_title_subsection`: STRING Example: "제1관 신의성실의무 등"
  - `metadata_title_supplementary`: STRING Available options: ['부칙 <제20305호,2024. 2. 13.>']
Relationsh

In [9]:
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
)

chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=enhanced_graph,
    verbose=True,
    use_function_response=True,
    function_response_system="당신은 자본시장법을 잘 알고 있는 법률 전문가입니다. 사용자의 질의에 graph database 내용을 기반으로 친절하고 자세하게 답변해주세요.",
)

### Add examples in the Cypher generation prompt

In [10]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Examples: Here are a few examples of generated Cypher statements for particular questions:
# **증권신고서** 또는 **정정신고서** 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 **배상책임**의 근거가 되는 조문은?
MATCH (l:law)
WHERE ANY(subtitle IN ['배상책임', '증권신고서', '정정신고서'] WHERE l.subtitle CONTAINS subtitle) OR ANY(content IN ['배상책임', '증권신고서', '정정신고서'] WHERE l.content CONTAINS content)
RETURN l
LIMIT 5

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)


chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=enhanced_graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    use_function_response=True,
    function_response_system="당신은 자본시장법을 잘 알고 있는 법률 전문가입니다. 반환된 문서 내용을 기반으로 친절하고 자세하게 답변해주세요.",
)

In [11]:
chain.invoke({"query": "**증권신고서**와 **투자설명서**의 **부실기재**에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?"})



> Entering new GraphCypherQAChain chain...


BadRequestError: Error code: 400 - {'error': {'message': "Your organization must qualify for at least usage tier 5 to access 'o1-preview'. See https://platform.openai.com/docs/guides/rate-limits/usage-tiers for more details on usage tiers.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'below_usage_tier'}}

### prompt in korean

In [7]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_KOREAN_TEMPLATE = """작업: 지시사항을 따라 그래프 데이터베이스를 조회하기 위한 Cypher 문을 생성하세요.
지시사항:
주어진 스키마의 관계 유형과 속성만 사용하세요.
제공되지 않은 관계 유형이나 속성은 사용하지 마세요.
스키마:
{schema}
참고: 응답에 설명이나 사과를 포함하지 마세요.
Cypher 문을 생성하는 것 이외의 질문에는 응답하지 마세요.
생성된 Cypher 문 이외의 텍스트는 포함하지 마세요.

예시: 다음은 특정 질문에 대해 생성된 Cypher 문 예시입니다:
# **증권신고서** 또는 **정정신고서** 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 **배상책임**의 근거가 되는 조문은?
MATCH (l:law)
WHERE ANY(subtitle IN ['배상책임', '증권신고서', '정정신고서'] WHERE l.subtitle CONTAINS subtitle) OR ANY(content IN ['배상책임', '증권신고서', '정정신고서'] WHERE l.content CONTAINS content)
RETURN l
LIMIT 5

질문:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_KOREAN_TEMPLATE
)


chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=enhanced_graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    use_function_response=True,
    function_response_system="당신은 자본시장법을 잘 알고 있는 법률 전문가입니다. 반환된 문서 내용을 기반으로 친절하고 자세하게 답변해주세요.",
)


In [8]:
chain.invoke({"query": "**증권신고서**와 **투자설명서**의 **부실기재**에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (l:law)
WHERE ANY(sub IN ['증권신고서', '투자설명서', '부실기재'] WHERE l.subtitle CONTAINS sub) OR ANY(cont IN ['증권신고서', '투자설명서', '부실기재'] WHERE l.content CONTAINS cont)
RETURN l
LIMIT 5
Full Context:
[{'l': {'metadata_document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )\n', 'metadata_source': '국가법령정보센터', 'metadata_title_chapter': '제4장 영업행위 규칙', 'metadata_revise_info': '법률 제20305호, 2024. 2. 13., 일부개정\n', 'metadata_title_doc': '제2편 금융투자업', 'subtitle': '금융투자업자의 업무위탁', 'index': '제42조제8항', 'metadata_title_section': '제1절 공통 영업행위 규칙', 'metadata_date': '시행 2024. 8. 14.', 'content': '금융투자업자는 제1항 본문에 따라 업무위탁을 한 내용을 「금융소비자 보호에 관한 법률」 제23조제1항에 따른 계약서류 및 제123조제1항에 따른 투자설명서(집합투자업자의 경우 제124조제2항제3호에 따른 간이투자설명서를 포함한다. 이하 제64조, 제86조 및 제93조에서 같다)에 기재하여야 하며, 투자자와 계약을 체결한 후에 업무위탁을 하거나 그 내용을 변경한 경우에는 이를 투자자에게 통보하여야 한다.<개정 2013. 5. 28., 2020. 3. 24.>', 'metadata_title_subsection': '제1관 신의성실의무 등'}}, {'l': {'metadata_document_title': '자본시장과 금융투자업에 관한 법률 ( 약

{'query': '**증권신고서**와 **투자설명서**의 **부실기재**에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?',
 'result': '**증권신고서**와 **투자설명서**의 **부실기재**에 대한 과징금 부과대상과 부과금액에 대해 설명드리겠습니다.\n\n### 1. 부과대상\n**증권신고서**와 **투자설명서**의 부실기재에 대한 과징금의 부과 대상은 다음과 같습니다:\n- **발행인**: 증권신고서 및 투자설명서를 작성하여 제출하는 주체로, 주로 회사가 해당됩니다.\n- **금융투자업자**: 투자설명서를 작성하거나 제출하는 역할을 하는 금융기관 등이 포함됩니다.\n\n### 2. 부과금액\n과징금의 부과금액은 다음과 같은 요소를 고려하여 결정됩니다:\n- **위반행위의 중대성**: 부실기재의 정도와 그로 인해 발생할 수 있는 투자자 피해의 규모 등이 영향을 미칩니다.\n- **위반행위의 횟수**: 동일한 또는 유사한 위반행위가 반복된 경우, 과징금이 가중될 수 있습니다.\n- **위반행위로 인한 이익**: 부실기재로 인해 얻은 이익의 규모도 과징금 산정에 반영될 수 있습니다.\n\n과징금의 구체적인 금액은 자본시장법 및 관련 시행령에서 정하는 바에 따릅니다. 법령에서는 일반적으로 과징금의 상한선을 설정하고 있으며, 구체적인 금액은 금융위원회나 관련 감독기관이 사안별로 결정하게 됩니다.\n\n### 관련 법조항\n- **자본시장법 제119조**: 증권신고서의 기재사항과 관련된 규정\n- **자본시장법 제123조**: 투자설명서의 기재사항과 관련된 규정\n- **자본시장법 제64조**: 금융투자업자의 손해배상책임과 관련된 규정\n\n부실기재로 인한 과징금은 투자자의 신뢰를 보호하고 시장의 투명성을 유지하기 위한 중요한 제재수단입니다. 따라서 정확한 기재와 성실한 정보 제공이 요구됩니다.\n\n추가적인 세부사항이 필요하시거나 법령 원문을 참고하고자 하신다면, 국가법령정보센터에서 자본시장법 관련 조항을 확인하실 수 있습니다.'}

# VLLM

> https://python.langchain.com/v0.2/docs/integrations/llms/vllm/

In [ ]:
%pip install --upgrade --quiet  vllm -q

## local model 구축 시

In [ ]:
from langchain_community.llms import VLLM

llm = VLLM(
    model="mosaicml/mpt-7b",
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=128,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

print(llm.invoke("What is the capital of France ?"))

from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Who was the US president in the year the first Pokemon game was released?"

print(llm_chain.invoke(question))

## awq quantization

In [ ]:
llm_q = VLLM(
    model="TheBloke/Llama-2-7b-Chat-AWQ",
    trust_remote_code=True,
    max_new_tokens=512,
    vllm_kwargs={"quantization": "awq"},
)

## openai

> https://python.langchain.com/v0.2/api_reference/community/llms/langchain_community.llms.vllm.VLLMOpenAI.html

In [ ]:
from langchain_community.llms import VLLMOpenAI

llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    model_name="gpt-4o",
    model_kwargs={"stop": ["."]},
)
print(llm.invoke("Rome is"))